# Hacking Dask clusters

# Inspecting the cluster's state

In [2]:
from dask.distributed import LocalCluster, Client, Worker

cluster = LocalCluster()
# cluster = LocalCluster(worker_class=Worker)
client = Client(cluster)

In [3]:
import dask.array as da

x = da.random.random((100, 100), chunks=(50, 50))
x = x.persist()

In [5]:
scheduler = cluster.scheduler
type(scheduler)

distributed.scheduler.Scheduler

Note that cluster manager classes won't _always_ have direct access to the `Scheduler` instance (e.g. when the cluster manager and scheduler are running on separate machines). We'll talk about how to inspect the scheduler's state in these cases later.

The scheduler tracks **a lot** of state. Let's start to explore the scheduler to get a sense for what information it keeps track of.

In [6]:
dict(scheduler.workers)

{'tcp://127.0.0.1:61799': <Worker 'tcp://127.0.0.1:61799', name: 2, memory: 1, processing: 0>,
 'tcp://127.0.0.1:61802': <Worker 'tcp://127.0.0.1:61802', name: 0, memory: 1, processing: 0>,
 'tcp://127.0.0.1:61805': <Worker 'tcp://127.0.0.1:61805', name: 3, memory: 1, processing: 0>,
 'tcp://127.0.0.1:61808': <Worker 'tcp://127.0.0.1:61808', name: 1, memory: 1, processing: 0>}

In [9]:
worker_state = next(iter(scheduler.workers.values()))
type(worker_state)

distributed.scheduler.WorkerState

Let's take a look at the `WorkerState` attributes

In [11]:
[attr for attr in dir(worker_state) if not attr.startswith("_")]

['actors',
 'address',
 'bandwidth',
 'clean',
 'executing',
 'extra',
 'has_what',
 'host',
 'identity',
 'last_seen',
 'local_directory',
 'memory_limit',
 'metrics',
 'name',
 'nanny',
 'nbytes',
 'ncores',
 'nthreads',
 'occupancy',
 'pid',
 'processing',
 'resources',
 'services',
 'status',
 'time_delay',
 'used_resources',
 'versions']

In [19]:
worker_state.address

'tcp://127.0.0.1:61799'

In [21]:
worker_state.status

<Status.running: 'running'>

In [22]:
worker_state.processing

{}

In [23]:
worker_state.last_seen

1618260932.8907268

In [28]:
import time

for _ in range(10):
    print(f"{worker_state.last_seen = }")
    time.sleep(0.5)

worker_state.last_seen = 1618261037.3917172
worker_state.last_seen = 1618261037.8874218
worker_state.last_seen = 1618261038.39227
worker_state.last_seen = 1618261038.88919
worker_state.last_seen = 1618261039.389277
worker_state.last_seen = 1618261039.8889751
worker_state.last_seen = 1618261040.388266
worker_state.last_seen = 1618261040.8895411
worker_state.last_seen = 1618261041.390165
worker_state.last_seen = 1618261041.8920572


In [16]:
worker_state.metrics

{'executing': 0,
 'in_memory': 1,
 'ready': 0,
 'in_flight': 0,
 'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}},
 'cpu': 4.4,
 'memory': 32690176,
 'time': 1618260849.389236,
 'read_bytes': 8174.082244835135,
 'write_bytes': 10217.602806043918,
 'num_fds': 29}

In [17]:
scheduler.total_nthreads

8

# Scheduler and worker plugins

# Coordination Primitives